<a href="https://colab.research.google.com/github/beyg1/Q4/blob/main/Quiz%20Practice/verbose_Agent_Runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [3]:
import nest_asyncio
nest_asyncio.apply()
#Agent loop doesnt run if dont make notebook capable of running async,
#because of AsyncOpenAI

In [12]:
from openai import AsyncOpenAI
from agents import Agent, Runner, OpenAIChatCompletionsModel, enable_verbose_stdout_logging, set_tracing_disabled, function_tool

set_tracing_disabled(True)
enable_verbose_stdout_logging()

external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

model = OpenAIChatCompletionsModel(
    model = "gemini-2.5-flash",
    openai_client = external_client
)

We can send 4 things to LLM.
1. System Prompt
2. User Prompt
3. Tool Schema
4. Tool Message

In [ ]:
# Error Case: No model settings provided since it's using external LLM
agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    #model=model  openai agents by default push for gpt-4o model no matter which LLM api you choose, so if you don't choose a model it will go for gpt-4o
    model = "gemini-2.0-flash", # this will give error too since openai_client is not set and can't pass that directly in Agent

)
response = Runner.run_sync(agent, "What is the meaning of life? answer in one sentence")
print(response.final_output)

In [10]:
# Working Case : Verbose enabled with simple calls
agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    model=model
)
response = Runner.run_sync(agent, "Hey, What's the weather in lahore")
print(response.final_output)

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7c75b3f133b0>


...

LLM resp:
{
  "content": "The weather in Lahore is currently **28°C** and it is **partly cloudy**. \n\nThe wind is blowing from the **West** at **9 km/h**. \n\nThe humidity is **69%**.\n\nFor the rest of the day, the forecast is for **partly cloudy skies** with a high of **32°C** and a low of **20°C**.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "The weather in Lahore is currently **28°C** and it is **partly cloudy**. \n\nThe wind is blowing from the **West** at **9 km/h**. \n\nThe humidity is **69%**.\n\nFor the rest of the day, the forecast is for **partly cloudy skies** with a high of **32°C** and a low of **20°C**.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


The weather in Lahore is currently **28°C** and it is **partly cloudy**. 

The wind is blowing from the **West** at **9 km/h**. 

The humidity is **69%**.

For the rest of the day, the forecast is for **partly cloudy skies** with a high of **32°C** and a low of **20°C**.


In [9]:
# Working Case : Verbose enabled with function calls
@function_tool
def get_current_weather(location: str) -> str:
   """Gets the current weather in a given location"""
   return f"The current weather in {location} is Sunny"

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    model=model,
    tools=[get_current_weather]
)
response = Runner.run_sync(agent, "Hey, What's the weather in lahore")
print(response.final_output)

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7c75b3f63bf0>


...

LLM resp:
{
  "content": "The current weather in lahore is Sunny.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "The current weather in lahore is Sunny.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


The current weather in lahore is Sunny.


In [11]:
# Working Case : Verbose enabled with function calls and max_turns
@function_tool
def get_current_weather(location: str) -> str:
   """Gets the current weather in a given location"""
   return f"The current weather in {location} is Sunny"

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    model=model,
    tools=[get_current_weather],
)
response = Runner.run_sync(agent, "Hey, What's the weather in lahore",max_turns=1)#This will give error for runner
print(response.final_output) # as max turns exceeds. usually max turns are "n +1". n being the function calls, LLM will have to perform.
# The tool call and tool output is returned in the first turn but requires second turn
# since the tool output will have to be put as input to LLM and then loop will run 1 more time.
# so in our case  Agent loop requires 2 turns

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7c75b3f118b0>


...

Tool get_current_weather returned The current weather in lahore is Sunny


DEBUG:openai.agents:Tool get_current_weather returned The current weather in lahore is Sunny


Resetting current trace


DEBUG:openai.agents:Resetting current trace


MaxTurnsExceeded: Max turns (1) exceeded

In [5]:
# Working Case : Playing with Agent constructor
agent = Agent(
    name="Assistant",
    #instructions="You are a helpful assistant.", #No instructions (works)
    model=model
)
response = Runner.run_sync(agent, "Hey, What's the weather in lahore")
print(response.final_output)

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7ba298df9850>


...

LLM resp:
{
  "content": "The weather in Lahore right now is **partly cloudy** with a temperature of **29°C**.\n\nHere are some more details:\n\n*   **Feels like:** 33°C\n*   **Wind:** 8 km/h from the South\n*   **Humidity:** 60%\n\nThe forecast for today suggests it will be **partly cloudy** with a high of **34°C** and a low of **23°C**.\n\nWould you like me to check the weather for a specific time or for tomorrow?",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "The weather in Lahore right now is **partly cloudy** with a temperature of **29°C**.\n\nHere are some more details:\n\n*   **Feels like:** 33°C\n*   **Wind:** 8 km/h from the South\n*   **Humidity:** 60%\n\nThe forecast for today suggests it will be **partly cloudy** with a high of **34°C** and a low of **23°C**.\n\nWould you like me to check the weather for a specific time or for tomorrow?",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


The weather in Lahore right now is **partly cloudy** with a temperature of **29°C**.

Here are some more details:

*   **Feels like:** 33°C
*   **Wind:** 8 km/h from the South
*   **Humidity:** 60%

The forecast for today suggests it will be **partly cloudy** with a high of **34°C** and a low of **23°C**.

Would you like me to check the weather for a specific time or for tomorrow?


In [6]:
# Error Case : Playing with Agent constructor
agent = Agent(
    #name="Assistant", No Name (don't work)
    #instructions="You are a helpful assistant.", #No instructions (works)
    model=model
)
response = Runner.run_sync(agent, "Hey, What's the weather in lahore")
print(response.final_output)

TypeError: Agent.__init__() missing 1 required positional argument: 'name'

In [8]:
# Working Case : Tweaking Agent constructor
agent = Agent(
    name=10, #integer passed as name (work's but python error comes as in Agent constructor name should be a str)
    instructions="You are a helpful assistant.",
    model=model
)
response = Runner.run_sync(agent, "Hey, What's the weather in lahore")
print(response.final_output)

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7ba29865d790>


...

LLM resp:
{
  "content": "The weather in Lahore is currently **24°C** and **cloudy**.\n\nThe wind is blowing from the **southwest** at **10 km/h**.\n\nThe humidity is **70%**.\n\nThe forecast for today predicts **light rain** with a high of **27°C** and a low of **20°C**.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "The weather in Lahore is currently **24°C** and **cloudy**.\n\nThe wind is blowing from the **southwest** at **10 km/h**.\n\nThe humidity is **70%**.\n\nThe forecast for today predicts **light rain** with a high of **27°C** and a low of **20°C**.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


The weather in Lahore is currently **24°C** and **cloudy**.

The wind is blowing from the **southwest** at **10 km/h**.

The humidity is **70%**.

The forecast for today predicts **light rain** with a high of **27°C** and a low of **20°C**.


In [15]:
# Working Case : Observing Synchronus vs Async behaviour
import time     # Synchronus

@function_tool
def task_one() -> str:
    """Performs the first task with a 5-second delay."""
    time.sleep(5)
    print("********Task One Completed********")
    return "Task One Completed"

@function_tool
def task_two() -> str:
    """Performs the second task with a 3-second delay."""
    time.sleep(3)
    print("********Task Two Completed********")
    return "Task Two Completed"

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    model=model,
    tools=[task_one, task_two]
)

response = Runner.run_sync(agent, "Perform Task One and then Task Two.",max_turns=3) #takes 3 turns as there are 2 tool calls.
print(response.final_output) # if max output is 2 then loop breaks and agent stops before generating response

Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee58299850>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee58299850>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee58299850>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee58299850>


...

LLM resp:
{
  "content": "Both tasks have been completed.\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}

LLM resp:
{
  "content": "Both tasks have been completed.\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}

LLM resp:
{
  "content": "Both tasks have been completed.\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}

LLM resp:
{
  "content": "Both tasks have been completed.\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "Both tasks have been completed.\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace
Resetting current trace
Resetting current trace
Resetting current trace


DEBUG:openai.agents:Resetting current trace


Both tasks have been completed.



In [17]:
# Working Case : Observing Synchronus vs Async behaviour
import time      # Synchronus 2nd type

@function_tool
def task_one() -> str:
    """Performs the first task with a 5-second delay."""
    time.sleep(5)
    print("********Task One Completed********")
    return "Task One Completed"

@function_tool
def task_two() -> str:
    """Performs the second task with a 3-second delay."""
    time.sleep(3)
    print("********Task Two Completed********")
    return "Task Two Completed"

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    model=model,
    tools=[task_one, task_two]
)

response = Runner.run_sync(agent, "Perform Task One and Task Two together.",max_turns=3) # let's prompt it to run both tools together
print(response.final_output) # responds it can only perform the task one by one. so it would still require 3 turns

Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee4b5b2f90>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee4b5b2f90>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee4b5b2f90>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee4b5b2f90>


...

LLM resp:
{
  "content": "I can perform Task One and Task Two sequentially for you. Would you like me to proceed with that?",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}

LLM resp:
{
  "content": "I can perform Task One and Task Two sequentially for you. Would you like me to proceed with that?",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}

LLM resp:
{
  "content": "I can perform Task One and Task Two sequentially for you. Would you like me to proceed with that?",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}

LLM resp:
{
  "content": "I can perform Task One and Task Two sequentially for you. Would you like me to proceed with that?",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": nu

DEBUG:openai.agents:LLM resp:
{
  "content": "I can perform Task One and Task Two sequentially for you. Would you like me to proceed with that?",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace
Resetting current trace
Resetting current trace
Resetting current trace


DEBUG:openai.agents:Resetting current trace


I can perform Task One and Task Two sequentially for you. Would you like me to proceed with that?


In [18]:
# Working Case : Observing Synchronus vs Async behaviour
import asyncio        # Async

@function_tool
async def task_one() -> str:
    """Performs the first task with a 5-second delay asynchronously."""
    await asyncio.sleep(5)
    return "Task One Completed"

@function_tool
async def task_two() -> str:
    """Performs the second task with a 3-second delay asynchronously."""
    await asyncio.sleep(3)
    return "Task Two Completed"

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    model=model,
    tools=[task_one, task_two]
)
async def main():
 response = await Runner.run(agent, "Perform Task One and then Task Two.")
 print(response.final_output)

asyncio.run(main())  # Even if it is Async Programming the agent loop took 3 turns
                     # also completed tasks synchronously even if it was Async function

Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee4b5931d0>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee4b5931d0>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee4b5931d0>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee4b5931d0>


...

LLM resp:
{
  "content": "Task One and Task Two have been performed.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}

LLM resp:
{
  "content": "Task One and Task Two have been performed.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}

LLM resp:
{
  "content": "Task One and Task Two have been performed.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}

LLM resp:
{
  "content": "Task One and Task Two have been performed.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "Task One and Task Two have been performed.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace
Resetting current trace
Resetting current trace
Resetting current trace


DEBUG:openai.agents:Resetting current trace


Task One and Task Two have been performed.


In [20]:
# Working Case : Observing Synchronus vs Async behaviour
import asyncio        # Async 2nd type

@function_tool
async def task_one() -> str:
    """Performs the first task with a 5-second delay ."""
    await asyncio.sleep(5)
    return "Task One Completed"

@function_tool
async def task_two() -> str:
    """Performs the second task with a 3-second delay ."""
    await asyncio.sleep(3)
    return "Task Two Completed"

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    model=model,
    tools=[task_one, task_two]
)
async def main():
 response = await Runner.run(agent, "Perform Task One and Task Two simultaneously.",max_turns=2)
 print(response.final_output)

asyncio.run(main()) #Both tasks have been completed in 2 turns when asked to complete them simultaneously

Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee4b33f1d0>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee4b33f1d0>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee4b33f1d0>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x79ee4b33f1d0>


...

LLM resp:
{
  "content": "Both Task One and Task Two have been completed.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}

LLM resp:
{
  "content": "Both Task One and Task Two have been completed.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}

LLM resp:
{
  "content": "Both Task One and Task Two have been completed.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}

LLM resp:
{
  "content": "Both Task One and Task Two have been completed.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "Both Task One and Task Two have been completed.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace
Resetting current trace
Resetting current trace
Resetting current trace


DEBUG:openai.agents:Resetting current trace


Both Task One and Task Two have been completed.
